In [1]:
%load_ext autoreload
%autoreload 2

#Imports from this projects
import auxiliary.util as util
util.set_wd_to_package_root()
import auxiliary.config as config
import auxiliary.grid2op_util as g2o_util
from auxiliary.generate_action_space import action_identificator
import data_preprocessing_analysis.imitation_data_preprocessing as idp

#Mathematics
import math
import numpy as np
from scipy.stats import entropy
from scipy.spatial import distance
from sklearn.metrics import confusion_matrix

#Collections
import collections
from collections import Counter

#File manipulation
import os
import json
from pathlib import Path

import pandas as pd #Data manipulation & analsysis
import grid2op #Grid simulation
import matplotlib.pyplot as plt #Plotting
import ipdb #Debugger
import re #Regular expressions
import functools #Higher-order functions
from tqdm import tqdm #Progress bar

In [2]:
config = config.get_config()

In [3]:
processed_data_path = config['paths']['processed_tutor_imitation']
con_matrix_path = config['paths']['con_matrix_cache']
fstats_path = config['paths']['feature_statistics']

line_disabled_to_consider = [-1,0,1,2,3,4,5,6,10,12,13,15,16,19]
line_group1 = [-1,0,1,2,3,4,5,6,12]
line_group2 = [13,15,16,19]
chronics_excluded = [310, 446, 777]

In [4]:
config['paths']['processed_tutor_imitation']

'data/processed_tutor_data/'

# Processed Data Analysis

The next block defines several data aggregates, such as counters. The processed data is loaded a file at a time, gradually filling the data aggregates.

In [5]:
n_sub = 14

#Inbstantiate the counter objects
counters = {}
for i in np.arange(-1,20):
    counters[i] = {
        'n_datapoints':0,
        'n_days_completed':0,
        'n_chronics':0,
        'set_hash': collections.Counter(),
        'res_hash': collections.Counter(),
        'tv_hash': collections.Counter(),
        'sub_changed': (n_sub+1) * [0],
        'changed_subs_n': n_sub * [0],
        'sub_info': []
    }
        
# To count numpy arrays, we index their counters with hashes, stored in hash dictionaries:
hash_to_act = {} #'Set'-action hashes
hash_to_tv = {} #Topology vector hashes
hash_to_res = {} #Resulting (i.e. post-action) topology vector hashes


for f in tqdm(list(Path(processed_data_path).rglob('*.json'))):
    with open(f, 'r') as file:
            dps = json.loads(file.read())
            
    line_disabled = dps[0]['line_disabled']
    
    counters[line_disabled]['n_chronics']+=1
    counters[line_disabled]['n_days_completed']+=dps[0]['dayscomp']
    for dp in dps:
        #Increase n. datapoints
        counters[line_disabled]['n_datapoints']+=1
        
        #Count set_topo_vect
        hsh_set = util.hash_nparray(np.array(dp['set_topo_vect']))
        if hsh_set not in hash_to_act:
            hash_to_act[hsh_set] = dp['set_topo_vect']
        counters[line_disabled]['set_hash'][hsh_set]+=1
        
        #Count res_topo_vect
        hsh_res = util.hash_nparray(np.array(dp['res_topo_vect']))
        if hsh_res not in hash_to_res:
            hash_to_res[hsh_res] = dp['res_topo_vect']
        counters[line_disabled]['res_hash'][hsh_res]+=1
        
        #Count topo_vect
        hsh_tv = util.hash_nparray(np.array(dp['topo_vect']))
        if hsh_tv not in hash_to_tv:
            hash_to_tv[hsh_tv] = dp['topo_vect']
        counters[line_disabled]['tv_hash'][hsh_tv]+=1
        
        #Count substations affected
        action_per_sub = g2o_util.tv_groupby_subst(dp['set_topo_vect'],dp['sub_info'])
        try:
            changed_subs_id = [np.any(a) for i,a in enumerate(action_per_sub)].index(True)
            counters[line_disabled]['sub_changed'][changed_subs_id] += 1
        except:
            counters[line_disabled]['sub_changed'][-1] += 1

        #Count topological depth of resulting topologies
        #ASSUMPTION: reference topology is the topology where all objects are connected to bus 1
        res_per_sub = g2o_util.tv_groupby_subst(dp['res_topo_vect'],dp['sub_info'])
        changed_subs_n = sum([2 in res for i,res in enumerate(res_per_sub)])
        counters[line_disabled]['changed_subs_n'][changed_subs_n] += 1
        
        #Set sub info
        counters[line_disabled]['sub_info'] = dp['sub_info']


100%|██████████| 997/997 [00:12<00:00, 81.22it/s] 


#### Number of chronics

In [6]:
sum([v['n_chronics']for k,v in counters.items() if k in line_disabled_to_consider])

997

#### Percentage days completed

In [7]:
print([(k,v['n_days_completed']/(28*v['n_chronics'])) if v['n_chronics']!=0 else None
       for k,v in counters.items() 
       if k in line_disabled_to_consider])

[(-1, 1.0), None, None, None, None, None, None, None, None, None, None, None, None, None]


In [8]:
print('Total:',
    sum([v['n_days_completed']for k,v in counters.items() if k in line_disabled_to_consider])/ \
    sum([28*v['n_chronics']for k,v in counters.items() if k in line_disabled_to_consider]))
print('Group 1:',
    sum([v['n_days_completed']for k,v in counters.items() if k in line_group1])/ \
    sum([28*v['n_chronics']for k,v in counters.items() if k in line_group1]))
print('Group 2:',
    sum([v['n_days_completed']for k,v in counters.items() if k in line_group2])/ \
    sum([28*v['n_chronics']for k,v in counters.items() if k in line_group2]))

Total: 1.0
Group 1: 1.0


ZeroDivisionError: division by zero

#### Number of datapoints

In [ ]:
[(k,v['n_datapoints'])for k,v in counters.items() if k in line_disabled_to_consider]

In [ ]:
N_total = sum([v['n_datapoints']for k,v in counters.items() if k in line_disabled_to_consider])
N_group1 = sum([v['n_datapoints']for k,v in counters.items() if k in line_group1])
N_group2 = sum([v['n_datapoints']for k,v in counters.items() if k in line_group2])

print('Total:', N_total)
print('Group 1:', N_group1)
print('Group 2:', N_group2)

In [ ]:
plt.bar(line_disabled_to_consider,
        [counters[l]['n_datapoints'] for l in np.arange(-1,20) if l in line_disabled_to_consider])
ax = plt.gca()
_ = ax.set_xticks(np.arange(-1,20))

#### Action statistics

Number of actions

In [ ]:
[(k,len(v['set_hash']))
 for k,v in counters.items() 
 if k in line_disabled_to_consider]

Percentage of do-nothing actions

In [ ]:
#Find the hashes that are do-nothing actions. Should not be higher than two. 
do_nothing_action_hashes = [h for h,t 
                            in hash_to_act.items() 
                            if sum(t)==0]
assert len(do_nothing_action_hashes) < 3

[(k,sum([v['set_hash'][h] for h in do_nothing_action_hashes])/v['n_datapoints'])
 for k,v in counters.items() 
 if k in line_disabled_to_consider and v['n_datapoints'] != 0]

In [ ]:
sum([sum([v['set_hash'][h] for h in do_nothing_action_hashes])
          /v['n_datapoints']
         for k,v in counters.items() 
         if k in line_disabled_to_consider and v['n_datapoints'] != 0])/N_total

In [ ]:
print('Total:',
     sum([sum([v['set_hash'][h] for h in do_nothing_action_hashes])
         for k,v in counters.items() 
         if k in line_disabled_to_consider and v['n_datapoints'] != 0])/N_total)
print('Group 1:',
     sum([sum([v['set_hash'][h] for h in do_nothing_action_hashes])
         for k,v in counters.items() 
         if k in line_group1 and v['n_datapoints'] != 0])/N_group1)
print('Group 2:',
     sum([sum([v['set_hash'][h] for h in do_nothing_action_hashes])
         for k,v in counters.items() 
         if k in line_group2 and v['n_datapoints'] != 0])/N_group2)

Entropy of the action distribution

In [ ]:
[(k,entropy(list(v['set_hash'].values()))) 
 for k,v in counters.items() 
 if k in line_disabled_to_consider]

In [ ]:
#Getting actions into format so that actions at substations
standard_sub_info = [3, 6, 4, 6, 5, 6, 3, 2, 5, 3, 3, 3, 4, 3]
    
for i in np.arange(-1,20):
    act_counter = counters[i]['set_hash']
    unique_act_counter = collections.Counter()
    for h,c in act_counter.items():
        a = hash_to_act[h]
        a_per_substation = g2o_util.tv_groupby_subst(a,counters[i]['sub_info'])
        try:
            changed_subs_id = [np.any(a) for a in a_per_substation].index(True)
            action = (changed_subs_id,tuple(a_per_substation[changed_subs_id]),
                      None if standard_sub_info[changed_subs_id]==counters[i]['sub_info'][changed_subs_id] else i)
            unique_act_counter[action] += c
        except ValueError:
            unique_act_counter[-1] += c
    counters[i]['unique_set_act'] = unique_act_counter
    
combined_act_counter = collections.Counter()
act_counter_group1 = collections.Counter()
act_counter_group2 = collections.Counter()
for i in np.arange(-1,20):
    combined_act_counter = combined_act_counter + counters[i]['unique_set_act']
    if i in line_group1:
        act_counter_group1 += act_counter_group1 + counters[i]['unique_set_act']
    elif  i in line_group2:
        act_counter_group2 += act_counter_group2 + counters[i]['unique_set_act']
        

In [ ]:
tups = list(combined_act_counter.keys())
tups = sorted([t for t in tups if type(t)==tuple])
tups = [-1] + tups
tups

colormap = {
    -1:'k',
    1:'b',
    2:'g',
    3:'r',
    4:'c',
    5:'m',
    8:'y',
    12:'k',
}

In [ ]:
fig, axs = plt.subplots(4,4,figsize=[2*6.4, 2*4.8])#, sharex=True, sharey=True)
axs = axs.reshape(-1)
fig.tight_layout()

for plt_i, c_i in enumerate(line_disabled_to_consider):
    act_counter = counters[c_i]['unique_set_act']
    weight = [act_counter[i] for i in tups]

    _, _, patches = axs[plt_i].hist(range(len(weight)), weights=weight,bins=range(len(weight)))
    axs[plt_i].title.set_text(f'Line {c_i} disabled.' if c_i>-1 else 'No line disabled.')
    
    #Applying colors
    for j,t in enumerate(tups[:-1]):
        if type(t) == int:
            continue
        patches[j].set_facecolor(colormap[t[0]])
    patches[0].set_facecolor(colormap[-1])
    
    axs[plt_i].ticklabel_format(axis="y", style="sci", scilimits=(0,0))

for i in range(len(axs)-len(line_disabled_to_consider)):
    axs[len(axs)-i-1].axis('off')
    
_ = fig.suptitle('Resulting \'Set\' Action Distribution per Topology', fontsize=16, y=1.05)
fig.savefig('data_preprocessing_analysis/figures/action_distribution_per_topology.png', dpi=300)

In [ ]:
counters[-1]['set_hash'].most_common()

In [ ]:
weight = [combined_act_counter[i] for i in tups]
_, _, patches = plt.hist(range(len(weight)), weights=weight,bins=range(len(weight)))

#Applying colors
for j,t in enumerate(tups[:-1]):
    if type(t) == int:
        continue
    patches[j].set_facecolor(colormap[t[0]])
patches[0].set_facecolor(colormap[-1])
plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

In [ ]:
weight = [c for v,c in counters[-1]['set_hash'].most_common()]
_, _, patches = plt.hist(range(len(weight)), weights=weight,bins=range(len(weight)))

#Applying colors
#patches[0].set_facecolor(colormap[-1])
#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

In [ ]:
print('Total:',
    len(combined_act_counter.most_common()),
      entropy(list(combined_act_counter.values())))
print('Group 1:',
      len(act_counter_group1.most_common()),
      entropy(list(act_counter_group1.values())))
print('Group 2:',
      len(act_counter_group2.most_common()),
      entropy(list(act_counter_group2.values())))

In [ ]:
plt.figure(figsize=[2*6.4, 2*4.8], dpi=80)

data = np.random.rand(len(line_disabled_to_consider), len(line_disabled_to_consider))
for iy,y in enumerate([counters[i]['unique_set_act'] for i in line_disabled_to_consider]):
    for ix,x in enumerate([counters[i]['unique_set_act'] for i in line_disabled_to_consider]):
        cosine = distance.cosine([y[a] for a in combined_act_counter.keys()],
                                     [x[a] for a in combined_act_counter.keys()])
        data[iy,ix] = cosine
heatmap = plt.pcolor(data,cmap='viridis_r')

for y in range(data.shape[0]):
    for x in range(data.shape[1]):
        plt.text(x + 0.5, y + 0.5, '%.4f' % data[y, x],
                 horizontalalignment='center',
                 verticalalignment='center',
                 )

plt.colorbar(heatmap)
plt.xticks(np.arange(0.5,len(line_disabled_to_consider)+0.5),['None']+line_disabled_to_consider[1:])
plt.yticks(np.arange(0.5,len(line_disabled_to_consider)+0.5),['None']+line_disabled_to_consider[1:])
plt.ylabel('Line disabled')
plt.xlabel('Line disabled')
plt.title('Cosine distance between the action distributions of the different topologies.')
plt.savefig('data_preprocessing_analysis/figures/cosine_distance_actions.png', dpi=300)
plt.show()

In [ ]:
(len(combined_act_counter.most_common()),entropy(list(combined_act_counter.values())))

### Topology Distribution

In [ ]:
fig, axs = plt.subplots(4,4,figsize=[2*6.4, 2*4.8])#, sharex=True, sharey=True)
axs = axs.reshape(-1)
fig.tight_layout()
for plt_i, c_i in enumerate(line_disabled_to_consider):
    res_counter = counters[c_i]['tv_hash']
    if not res_counter:
        continue
    val, weight = zip(*[(i, v) for i,(k,v) in enumerate(res_counter.most_common())])
    axs[plt_i].hist(val[0:100], weights=weight[0:100],bins=val[0:100])
    axs[plt_i].title.set_text(f'Line {c_i} removed')

for i in range(len(axs)-len(line_disabled_to_consider)):
    axs[len(axs)-i-1].axis('off')
    
_ = fig.suptitle('Topology vector distribution per line outage', fontsize=16, y=1.05)

In [ ]:
fig, axs = plt.subplots(4,4,figsize=[2*6.4, 2*4.8])#, sharex=True, sharey=True)
axs = axs.reshape(-1)
fig.tight_layout()
for plt_i, c_i in enumerate(line_disabled_to_consider):
    res_counter = counters[c_i]['res_hash']
    if not res_counter:
        continue
    val, weight = zip(*[(i, v) for i,(k,v) in enumerate(res_counter.most_common())])
    axs[plt_i].hist(val[0:100], weights=weight[0:100],bins=val[0:100])
    axs[plt_i].title.set_text(f'Line {c_i} removed')

for i in range(len(axs)-len(line_disabled_to_consider)):
    axs[len(axs)-i-1].axis('off')
    
_ = fig.suptitle('Resulting topology distribution per line outage', fontsize=16, y=1.05)

In [ ]:
counters[-1]['res_hash'].most_common()

### Substations acted on

In [ ]:
substations_with_actions = [1,2,3,4,5,8,12,-1]

In [ ]:
np.array(counters[c_i]['sub_changed'])[substations_with_actions]

In [ ]:
fig, axs = plt.subplots(4,4,figsize=[2*6.4, 2*4.8])#, sharex=True, sharey=True)
axs = axs.reshape(-1)
fig.tight_layout()
for plt_i, c_i in enumerate(line_disabled_to_consider):
    axs[plt_i].bar([str(b) for b in substations_with_actions],
                   np.array(counters[c_i]['sub_changed'])[substations_with_actions])
    axs[plt_i].title.set_text(f'Line {c_i} removed')

for i in range(len(axs)-len(line_disabled_to_consider)):
    axs[len(axs)-i-1].axis('off')
    
_ = fig.suptitle('Substations acted on per line outage', fontsize=16, y=1.05)

In [ ]:
fig, axs = plt.subplots(1,3,figsize=[3*6.4, 1*4.8])

axs[0].bar([str(l) for l in substations_with_actions[:-1]] + ['No action'],
        np.sum(np.array([counters[c_i]['sub_changed'] \
                                                   for c_i in line_group1]),axis=0)[substations_with_actions])
axs[0].ticklabel_format(axis="y", style="sci", scilimits=(0,0))
axs[0].set_title('Cluster 1')

axs[1].bar([str(l) for l in substations_with_actions[:-1]] + ['No action'],
        np.sum(np.array([counters[c_i]['sub_changed'] \
                                                   for c_i in line_group2]),axis=0)[substations_with_actions])
axs[1].ticklabel_format(axis="y", style="sci", scilimits=(0,0))
axs[1].set_title('Cluster 2')

axs[2].bar([str(l) for l in substations_with_actions[:-1]] + ['No action'],
        np.sum(np.array([counters[c_i]['sub_changed'] \
                                                   for c_i in line_disabled_to_consider]),axis=0)[substations_with_actions])
axs[2].ticklabel_format(axis="y", style="sci", scilimits=(0,0))
axs[2].set_title('Total')

fig.suptitle('Distribution of Substations Acted On', fontsize=16, y=1)
fig.savefig('data_preprocessing_analysis/figures/distribution_substations.png', dpi=300)

In [ ]:
plt.bar([str(l) for l in substations_with_actions[:-1]] + ['No action'],
        np.sum(np.array([counters[c_i]['sub_changed'] \
                                                   for c_i in line_disabled_to_consider]),axis=0)[substations_with_actions])
#plt.title('Substations Acted On')

### Topological Depth

In [ ]:
do_nothing_action_hashes = [h for h,t in hash_to_act.items() if sum(t)==0]
def mean_index(lst):
    return np.sum(np.array([i*v for i,v in enumerate(lst)]))/sum(lst)
[(k,mean_index(v['changed_subs_n']))
 for k,v in counters.items() if k in line_disabled_to_consider]

mean topological depth

In [ ]:
print('Total:',
    mean_index(np.sum(np.array([v['changed_subs_n'] for k,v in counters.items() 
                            if k in line_disabled_to_consider ]),axis=0)))
print('Group 1:',
      mean_index(np.sum(np.array([v['changed_subs_n'] for k,v in counters.items() 
                            if k in line_group1 ]),axis=0)))
print('Group 2:',
    mean_index(np.sum(np.array([v['changed_subs_n'] for k,v in counters.items() 
                            if k in line_group2 ]),axis=0)))

In [ ]:
fig, axs = plt.subplots(4,4,figsize=[2*6.4, 2*4.8])#, sharex=True, sharey=True)
axs = axs.reshape(-1)
fig.tight_layout()
for plt_i, c_i in enumerate(line_disabled_to_consider):
    axs[plt_i].bar([str(n) for n in range(len(substations_with_actions))],
                   counters[c_i]['changed_subs_n'][0:len(substations_with_actions)])
    axs[plt_i].title.set_text(f'Line {c_i} removed')

for i in range(len(axs)-len(line_disabled_to_consider)):
    axs[len(axs)-i-1].axis('off')
    
_ = fig.suptitle('Topological depth of resulting topologies per line outage', fontsize=16, y=1.05)

In [ ]:
plt.bar([str(n) for n in range(n_sub)],np.sum(np.array([counters[c_i]['changed_subs_n'] \
                                                        for c_i in np.arange(-1,20)]),axis=0))
plt.title('')

In [ ]:
fig, axs = plt.subplots(1,3,figsize=[3*6.4, 1*4.8])

axs[0].bar(range(len(substations_with_actions)),
        np.sum(np.array([counters[c_i]['changed_subs_n'] \
                                                   for c_i in line_group1]),axis=0)[0:8])
axs[0].ticklabel_format(axis="y", style="sci", scilimits=(0,0))
axs[0].set_title('Cluster 1')

axs[1].bar(range(len(substations_with_actions)),
        np.sum(np.array([counters[c_i]['changed_subs_n'] \
                                                   for c_i in line_group2]),axis=0)[0:8])
axs[1].ticklabel_format(axis="y", style="sci", scilimits=(0,0))
axs[1].set_title('Cluster 2')

axs[2].bar(range(len(substations_with_actions)),
        np.sum(np.array([counters[c_i]['changed_subs_n'] \
                                                   for c_i in line_disabled_to_consider]),axis=0)[0:8])
axs[2].ticklabel_format(axis="y", style="sci", scilimits=(0,0))
axs[2].set_title('Total')


fig.suptitle('Distribution of Topological Depth of Actions', fontsize=16, y=1)
fig.savefig('data_preprocessing_analysis/figures/distribution_topological_depth.png', dpi=300)
#TODO: remove susbtations that never have any actions